
# Microsoft Fabric - Bulk Export All Items to Local Files using Fabric CLI (Python)

This notebook uses the official Microsoft Fabric CLI (`fab`) from Python to export every exportable item from one or more workspaces.
- Idea for this based off the blog post: https://peerinsights.emono.dk/fabric-cli-beyond-shell-commands?s=03

In [ ]:
#########################
## Input Parameters
#########################

# This is the Workspace Name for the items you want to download - NOTE: It has got the .Workspace at the end as this is required for the Fabric CLI
target_workspaceName = "FILL ME IN.Workspace"

# This is the Workspace GUID
target_workspace_id = "FILL ME IN" 

# Lakehouse Name
target_lakehouse = "FILL ME IN"          

# Lakehouse GUID
target_lakehouse_id = "FILL ME IN"

In [2]:
# Cell 1: Install Fabric CLI (only needed once per environment)
import sys
!{sys.executable} -m pip install --quiet ms-fabric-cli --upgrade
print("Fabric CLI installed/updated")

Fabric CLI installed/updated


In [ ]:
######################################################################################### 
# Read secretes from Azure Key Vault
#########################################################################################
## This is the name of my Azure Key Vault 
key_vault = "https://FILL ME IN.vault.azure.net/"
## I have stored my tenant id as one of the secrets to make it easier to use when needed 
tenant = notebookutils.credentials.getSecret(key_vault , "FILL ME IN") 
## This is my application Id for my service principal account 
client = notebookutils.credentials.getSecret(key_vault , "FILL ME IN") 
## This is my Client Secret for my service principal account 
client_secret = notebookutils.credentials.getSecret(key_vault , "FILL ME IN")


######################################################################################### 
# Authentication - Replace string variables with your relevant values 
#########################################################################################  

import json, requests, pandas as pd 
import datetime  

try: 
    from azure.identity import ClientSecretCredential 
except Exception:
     !pip install azure.identity 
     from azure.identity import ClientSecretCredential 

# Generates the access token for the Service Principal 
api = 'https://analysis.windows.net/powerbi/api/.default' 
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/', 
               tenant_id = tenant, 
               client_id = client, 
               client_secret = client_secret) 
access_token = auth.get_token(api)
access_token = access_token.token 

## This is where I store my header with the Access Token, because this is required when authenticating 
## to the Power BI Admin APIs 
header = {'Authorization': f'Bearer {access_token}'}  

print('\nSuccessfully authenticated.')


Successfully authenticated.


## Begin Login to Fabric CLI

In [4]:
!fab config set encryption_fallback_enabled true

In [ ]:

from argparse import Namespace
from fabric_cli.commands.config import fab_config
from fabric_cli.commands.auth import fab_auth

# Login using service principal
args = Namespace(
    auth_command="login",
    username=client,
    password=client_secret,
    tenant=tenant,
    identity=None,
    federated_token=None,
    certificate=None
)
fab_auth.init(args)
fab_auth.status(None)  # Check current authentication status


In [6]:
# Cell 3: List all your workspaces (helps you pick the ones you want)
# !fab ls /Fabric_FourMoo.Workspace

## Get all Items and create a Dataframe

In [ ]:
import pandas as pd
import subprocess

# Capture the command output
result = subprocess.run(
    ['fab', 'ls', f"/{target_workspaceName}"],
    capture_output=True,
    text=True
)

# Get the output
output = result.stdout

# Parse the ls -la output
lines = output.strip().split('\n')

# Skip the first line (total) if present
if lines[0].startswith('total'):
    lines = lines[1:]

data = []
for line in lines:
        data.append({
            'item_name': line
        })

# Create DataFrame
df = pd.DataFrame(data)
print(df)

In [8]:
display(df)

In [12]:
##################################
## Exporting to OneLake
##################################


# OneLake path template (Files is the default for uploads)
onelake_base = f"abfss://{target_workspace_id}@onelake.dfs.fabric.microsoft.com/{target_lakehouse_id}/Files"

all_usage_dfs = []

oneLake_export_folder_name = "AllItemsExport"

# Loop through each row and collect data
for row in df.itertuples():
    try:
        # Assign variables from row
        var_ItemName = str(row.item_name).strip()   # make sure no trailing spaces

        print(var_ItemName)
    
        # ←←← THESE ARE THE ONLY LINES YOU ASKED TO CHANGE
        workspace_name = target_workspaceName          # your workspace
        item_name      = var_ItemName                        # ← use the name from the row

        # Where to save it (local temp → OneLake)
        import tempfile, os
        from datetime import datetime

        with tempfile.TemporaryDirectory() as tmp:
            # Create a folder inside the temp directory that matches the item name
            item_output_folder = f"{tmp}/{oneLake_export_folder_name}/{var_ItemName}"
            os.makedirs(item_output_folder, exist_ok=True)
            
            full_item_path = f"/{workspace_name}/{item_name}"
            
            print(f"Exporting: {full_item_path}")
            !fab export "{full_item_path}" -f -o "{item_output_folder}"
            
            # Copy the whole item folder (not just the AllItemsExport root) to OneLake
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            onelake_path = f"{onelake_base}/{oneLake_export_folder_name}/{var_ItemName}_{timestamp}"
            print(f"OneLakePath={onelake_path}")
            
            notebookutils.fs.mkdirs(onelake_path)
            notebookutils.fs.cp(f"file:{item_output_folder}", onelake_path, recurse=True)
            
            # print(f"\nDone! Item exported to OneLake:")
            # displayHTML(f'<h3><a href="{onelake_path}" target="_blank">Open {var_ItemName} in OneLake</a></h3>')

    except Exception as e:
        print(f"Error exporting item '{var_ItemName}': {e}")

In [ ]:
# #####################################
# ## Export to Azure Storage Account
# #####################################
%pip install azure-identity azure-storage-blob azure-storage-file-datalake --quiet

# Import libraries
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
import tempfile
import os
from datetime import datetime

# Storage Details
storage_account_name = "FILL ME IN"
container_name       = "FILL ME IN"

# Service Principal Credentials (replace or use secrets)
TENANT_ID = tenant
CLIENT_ID = client
CLIENT_SECRET = client_secret

# Create credential
credential = ClientSecretCredential(
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

# Use DataLakeServiceClient (correct for ADLS Gen2 directories)
account_url = f"https://{storage_account_name}.dfs.core.windows.net"  # Note: .dfs not .blob
service_client = DataLakeServiceClient(account_url=account_url, credential=credential)

# Get file system client (equivalent to container)
file_system_client = service_client.get_file_system_client(file_system=container_name)

# Ensure the container exists (optional, will fail gracefully if not)
try:
    file_system_client.create_file_system()  # Idempotent
except Exception:
    pass  # Already exists

# ──────────────────────────────────────────────
# MAIN LOOP
# ──────────────────────────────────────────────
for row in df.itertuples():
    try:
        item_name = str(row.item_name).strip()
        print(f"\nProcessing: {item_name}")

        # 1. Create temporary local folder for this item
        with tempfile.TemporaryDirectory() as tmp_dir:
            local_folder = os.path.join(tmp_dir, "export", item_name)
            os.makedirs(local_folder, exist_ok=True)

            # 2. Export the Fabric item
            full_item_path = f"/{target_workspaceName}/{item_name}"
            print(f"Exporting {full_item_path} → {local_folder}")
            !fab export "{full_item_path}" -f -o "{local_folder}"

            # 3. Upload everything (skip junk files)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            remote_prefix = f"{item_name}_{timestamp}/"

            uploaded_count = 0
            for root, _, files in os.walk(local_folder):
                for f in files:
                    local_path = os.path.join(root, f)
                    file_size  = os.path.getsize(local_path)

                    # ── SKIP UNWANTED FILES ─────────────────────
                    if (not f.strip() or                     # empty filename
                        file_size == 0 or                     # zero-byte file
                        f.startswith(".") or                  # hidden files
                        f in {"__init__.py", ".gitkeep", "desktop.ini"}):
                        print(f"  Skipping junk file: {f or '<empty name>'}")
                        continue
                    # ─────────────────────────────────────────────

                    relative_path = os.path.relpath(local_path, local_folder).replace("\\", "/")
                    blob_path = remote_prefix + relative_path

                    print(f"  Uploading {blob_path} ({file_size} bytes)")
                    file_client = file_system_client.get_file_client(blob_path)
                    with open(local_path, "rb") as data:
                        file_client.upload_data(data, overwrite=True)
                    uploaded_count += 1

            print(f"Successfully uploaded {item_name} → {container_name}/{remote_prefix} ({uploaded_count} files)")

    except Exception as e:
        print(f"Error exporting/uploading '{item_name}': {str(e)}")



In [ ]:
# Cell 3 – EXPORT A SINGLE ITEM (change these two lines only)

# ←←← CHANGE THESE TWO LINES ONLY
workspace_name = "Fabric_FourMoo.Workspace"          # your workspace
item_name      = "Blog - Get All Fabric Items - Spark.Notebook "           # exact name of the item (with suffix)

# Where to save it (local temp → OneLake)
import tempfile, os
from datetime import datetime

with tempfile.TemporaryDirectory() as tmp:
    output_folder = f"{tmp}/SingleItemExport"
    os.makedirs(output_folder, exist_ok=True)
    
    full_item_path = f"/{workspace_name}/{item_name}"
    
    print(f"Exporting: {full_item_path}")
    !fab export "{full_item_path}" -f -o "{output_folder}"
    
    # Optional: copy straight to OneLake (Files section of the attached lakehouse)
    onelake_path = f"{onelake_base}/SingleItemExports/{item_name}_{datetime.now():%Y%m%d_%H%M%S}"
    notebookutils.fs.mkdirs(onelake_path)
    notebookutils.fs.cp(f"file:{output_folder}", onelake_path, recurse=True)
    
    print(f"\nDone! Item exported to OneLake:")
    displayHTML(f'<h3><a href="{onelake_path}" target="_blank">Open {item_name} in OneLake</a></h3>')